## Reference

[1] [超簡單台股每日爬蟲教學](https://www.finlab.tw/%E8%B6%85%E7%B0%A1%E5%96%AE%E5%8F%B0%E8%82%A1%E6%AF%8F%E6%97%A5%E7%88%AC%E8%9F%B2%E6%95%99%E5%AD%B8/)

***

# 建立一個 sqlite 檔案，整理抓到的 csv 檔案


 - Part I: 利用 glob 以及 pandas 套件來讀取我們抓來的 2014 - 2019 年的台股資料
 - Part II: 建立一個 sqlite 檔案來存入我們的資料庫
 - Part III: 進一步整理為用 "個股" 來區分的資料庫

In [1]:
# import package
import pandas as pd
import sqlite3
import glob

### Part I: 利用 glob 以及 pandas 套件來讀取我們抓來的 2014 - 2019 年的台股資料

 - 當我們用EP1的程式抓取大量的表單之後
 
 - 如果我們簡單用迴圈加 to_csv 方法，會發現現在我們的資料夾裡面已經堆滿的大量的 csv 檔案了
 
 - 我們要如何使用程式來整理我們的檔案勒？
 
 - 如果用 pandas 一個一個讀入顯然很沒有效率
 
 - 我們使用 **glob** 套件 **一次讀入所有的 csv 檔案**

In [2]:
'''
glob套件是用來查找 '符合特定規則' 的文件名，跟我們用搜尋跳出來的結果差不多，這邊我們查詢副檔名為csv的檔案並存為一個列表的形式。
'''

All_csv_file = glob.glob('*.csv')
# 因為當初儲存時，也將 index 存進去了，所以第一個 column 就省略，看第一個 column 以後的資料即可
df = pd.read_csv(All_csv_file[0]).iloc[:,1:]

In [3]:
All_csv_file

['20190522.csv',
 '20190523.csv',
 '20190521.csv',
 '20190509.csv',
 '20190508.csv',
 '20190520.csv',
 '20190524.csv',
 '20190530.csv',
 '20190531.csv',
 '20190527.csv',
 '20190620.csv',
 '20190621.csv',
 '20190619.csv',
 '20190625.csv',
 '20190624.csv',
 '20190618.csv',
 '20190626.csv',
 '20190627.csv',
 '20190617.csv',
 '20190603.csv',
 '20190628.csv',
 '20190614.csv',
 '20190610.csv',
 '20190604.csv',
 '20190605.csv',
 '20190611.csv',
 '20190613.csv',
 '20190612.csv',
 '20190606.csv',
 '20190517.csv',
 '20190701.csv',
 '20190516.csv',
 '20190514.csv',
 '20190528.csv',
 '20190702.csv',
 '20190703.csv',
 '20190529.csv',
 '20190515.csv',
 '20190510.csv',
 '20190506.csv',
 '20190507.csv',
 '20190513.csv']

In [4]:
df_tmp = pd.read_csv('20190701.csv')
df_tmp.head() 

,Unnamed: 0,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0,0050,元大台灣50,"13,771,395","5,806","1,137,417,652",82.05,82.80,82.05,82.65,+,1.75,82.65,125,82.70,408,0.00
1,1,0051,元大中型100,"48,263",27,"1,575,525",32.60,32.66,32.60,32.66,+,0.56,32.66,19,32.69,2,0.00
2,2,0052,富邦科技,"115,100",18,"6,320,374",54.20,54.95,54.20,54.95,+,1.85,54.95,3,55.00,6,0.00
3,3,0053,元大電子,"97,000",21,"3,401,140",34.69,35.15,34.69,35.10,+,1.09,35.03,2,35.10,1,0.00
4,4,0054,元大台商50,"35,023",17,"790,253",22.31,22.67,22.31,22.67,+,0.36,22.67,1,22.68,1,0.00


In [5]:
# 因為當初儲存時，也將 index 存進去了，所以第一個 column 就省略，看第一個 column 以後的資料即可
df_tmp.iloc[:,1:].head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"13,771,395","5,806","1,137,417,652",82.05,82.80,82.05,82.65,+,1.75,82.65,125,82.70,408,0.00
1,0051,元大中型100,"48,263",27,"1,575,525",32.60,32.66,32.60,32.66,+,0.56,32.66,19,32.69,2,0.00
2,0052,富邦科技,"115,100",18,"6,320,374",54.20,54.95,54.20,54.95,+,1.85,54.95,3,55.00,6,0.00
3,0053,元大電子,"97,000",21,"3,401,140",34.69,35.15,34.69,35.10,+,1.09,35.03,2,35.10,1,0.00
4,0054,元大台商50,"35,023",17,"790,253",22.31,22.67,22.31,22.67,+,0.36,22.67,1,22.68,1,0.00


### Part II: 建立一個 sqlite 檔案來存入我們的資料庫

 - python內建sqlite套件，我們無須特安裝
 
 - 支援完整sql語法查詢我們的資料
 
 - 使用以及轉移方便，一個資料庫就像一個本地文件一樣
 
 - 直接使用 DataFrame 整理成 SQL 資料庫
 

In [6]:
dbname = 'TWstock.db'
db = sqlite3.connect(dbname)

In [7]:
All_csv_file

['20190522.csv',
 '20190523.csv',
 '20190521.csv',
 '20190509.csv',
 '20190508.csv',
 '20190520.csv',
 '20190524.csv',
 '20190530.csv',
 '20190531.csv',
 '20190527.csv',
 '20190620.csv',
 '20190621.csv',
 '20190619.csv',
 '20190625.csv',
 '20190624.csv',
 '20190618.csv',
 '20190626.csv',
 '20190627.csv',
 '20190617.csv',
 '20190603.csv',
 '20190628.csv',
 '20190614.csv',
 '20190610.csv',
 '20190604.csv',
 '20190605.csv',
 '20190611.csv',
 '20190613.csv',
 '20190612.csv',
 '20190606.csv',
 '20190517.csv',
 '20190701.csv',
 '20190516.csv',
 '20190514.csv',
 '20190528.csv',
 '20190702.csv',
 '20190703.csv',
 '20190529.csv',
 '20190515.csv',
 '20190510.csv',
 '20190506.csv',
 '20190507.csv',
 '20190513.csv']

In [8]:
%%time
# read_csv -> data.frame -> to.sql -> db
for file_name in All_csv_file:
    pd.read_csv(file_name).iloc[:,1:].to_sql(file_name.replace('.csv',''), db , if_exists = 'replace')

CPU times: user 1.27 s, sys: 357 ms, total: 1.63 s
Wall time: 2.11 s


### 如何讀取資料庫的表格

 - 如何讀取 sqlite 檔案裡面的表格

In [9]:
dates_list = [file_name.replace('.csv','') for file_name in All_csv_file]

In [24]:
dates_list

['20190506',
 '20190507',
 '20190508',
 '20190509',
 '20190510',
 '20190513',
 '20190514',
 '20190515',
 '20190516',
 '20190517',
 '20190520',
 '20190521',
 '20190522',
 '20190523',
 '20190524',
 '20190527',
 '20190528',
 '20190529',
 '20190530',
 '20190531',
 '20190603',
 '20190604',
 '20190605',
 '20190606',
 '20190610',
 '20190611',
 '20190612',
 '20190613',
 '20190614',
 '20190617',
 '20190618',
 '20190619',
 '20190620',
 '20190621',
 '20190624',
 '20190625',
 '20190626',
 '20190627',
 '20190628',
 '20190701',
 '20190702',
 '20190703']

In [11]:
# show the data.frame from pd.read_sql
# for example: dates_list[0] = 20190703
pd.read_sql(con = db, sql = 'SELECT * FROM' + ' "'+ dates_list[0] +'"').iloc[:,1:].head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"10,557,991","4,582","827,306,010",78.20,78.65,78.10,78.40,,0.00,78.40,194,78.45,825,0.00
1,0051,元大中型100,"7,000",7,"217,620",31.05,31.12,31.00,31.00,+,0.04,30.95,4,31.00,31,0.00
2,0052,富邦科技,"111,000",6,"5,693,350",50.85,51.30,50.85,51.25,+,0.70,50.80,50,51.15,5,0.00
3,0053,元大電子,"8,000",7,"265,040",33.25,33.25,32.98,33.11,+,0.22,32.86,26,33.00,30,0.00
4,0054,元大台商50,"13,000",3,"274,590",21.16,21.16,20.99,20.99,-,0.05,20.99,1,21.06,60,0.00


### 第三部份 整理出以個股為一張張表的資料庫

In [103]:
# For example
date = '20190701'
df = pd.read_sql( con = db, sql = 'SELECT * FROM' + ' "'+ date +'"')
df['Date'] = date
total_df = total_df.append(df)

In [104]:
total_df.tail()

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
1092,1092,9944,新麗,"532,120",32,"9,287,906",17.50,17.55,17.45,17.50,+,0.05,17.40,5,17.55,15,145.83,20190701
1093,1093,9945,潤泰新,"929,149",589,"41,457,019",44.50,44.90,44.50,44.55,+,0.15,44.55,21,44.60,11,4.71,20190701
1094,1094,9946,三發地產,"795,270",371,"16,463,509",20.85,20.85,20.60,20.70,-,0.05,20.65,56,20.70,17,7.09,20190701
1095,1095,9955,佳龍,"4,388,151","1,436","89,177,811",20.30,20.60,20.30,20.30,-,2.25,--,0,20.30,"2,258",0.00,20190701
1096,1096,9958,世紀鋼,"962,616",579,"62,860,840",65.70,66.00,65.00,65.10,+,0.20,65.10,37,65.30,5,382.94,20190701


In [127]:
%%time
import time

total_df = pd.DataFrame()
for date in dates_list:
    time.sleep(1) # 由於過度頻繁撈取，使得撈不到資料，所以讓他慢一秒鐘去抓取可能會比較好。
    df = pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ date +'"')
    df['Date'] = date
    total_df = total_df.append(df)

CPU times: user 2.63 s, sys: 205 ms, total: 2.84 s
Wall time: 45 s


In [128]:
%%time
total_df.shape

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.8 µs


(47243, 18)

In [130]:
dbname_2 = 'TWstock_2'
db2 = sqlite3.connect(dbname_2)

In [131]:
%%time
total_dict = dict(tuple(total_df.groupby('證券代號')))

CPU times: user 300 ms, sys: 8.85 ms, total: 308 ms
Wall time: 309 ms


In [132]:
%%time
# For example dict's key insert: 2330
total_dict['2330'].sort_values(["Date"], axis=0, ascending=True) 

CPU times: user 1.21 ms, sys: 10 µs, total: 1.22 ms
Wall time: 1.22 ms


,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
409,409,2330,台積電,"33,688,187","12,535","8,717,053,470",260.00,260.00,258.00,259.00,-,6.0,259.00,397,259.50,555,19.13,20190506
409,409,2330,台積電,"25,686,126","8,339","6,719,199,455",259.50,263.00,259.00,262.50,+,3.5,262.50,33,263.00,854,19.39,20190507
409,409,2330,台積電,"25,902,364","7,905","6,750,083,504",260.00,261.50,259.50,260.00,-,2.5,260.00,"3,639",260.50,51,19.20,20190508
409,409,2330,台積電,"34,166,574","10,995","8,798,140,796",259.50,259.50,256.00,256.50,-,3.5,256.50,396,257.00,6,18.94,20190509
409,409,2330,台積電,"18,868,212","8,009","4,844,594,484",257.00,259.00,255.00,256.00,-,0.5,256.00,"2,305",256.50,27,18.91,20190510
409,409,2330,台積電,"29,317,535","11,974","7,374,249,589",253.00,254.00,249.50,250.50,-,5.5,250.50,224,251.00,510,18.50,20190513
409,409,2330,台積電,"45,620,708","14,283","11,345,067,455",247.50,251.00,245.00,248.50,-,2.0,248.50,241,249.00,7,18.35,20190514
409,409,2330,台積電,"37,223,479","12,041","9,327,051,888",251.00,252.00,249.00,249.00,+,0.5,249.00,"1,363",249.50,5,20.00,20190515
410,410,2330,台積電,"30,331,519","11,402","7,520,908,612",248.50,249.50,246.00,247.00,-,2.0,247.00,480,247.50,90,19.84,20190516
410,410,2330,台積電,"40,006,856","15,217","9,775,372,898",249.00,249.00,241.50,241.50,-,5.5,241.50,"1,899",242.00,65,19.40,20190517


In [121]:
%%time 
for key in total_dict.keys():
    df = total_dict[key].iloc[:,2:]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Date'])
    df.to_sql(key,db2,if_exists='replace')

CPU times: user 8.9 s, sys: 2.82 s, total: 11.7 s
Wall time: 15.2 s


In [125]:
%%time
#測試股票代號資料庫
pd.read_sql(con=db2,sql='SELECT * FROM "2330"').tail()

CPU times: user 6.96 ms, sys: 1.65 ms, total: 8.61 ms
Wall time: 7.71 ms


,index,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
38,409,台積電,"28,085,212","8,987","6,715,488,371",241.50,241.50,238.00,239.00,-,1.5,238.50,380,239.00,641,19.20,2019-06-28 00:00:00
39,409,台積電,"66,370,875","29,551","16,445,222,750",245.50,250.00,245.00,248.50,+,9.5,248.00,342,248.50,178,19.96,2019-07-01 00:00:00
40,409,台積電,"66,370,875","29,551","16,445,222,750",245.50,250.00,245.00,248.50,+,9.5,248.00,342,248.50,178,19.96,2019-07-01 00:00:00
41,409,台積電,"26,832,202","12,531","6,671,951,743",249.50,250.00,247.50,249.00,+,0.5,248.50,73,249.00,"3,369",20.00,2019-07-02 00:00:00
42,409,台積電,"29,549,774","16,033","7,190,777,082",244.00,245.00,242.00,242.50,-,6.5,242.50,489,243.00,292,19.48,2019-07-03 00:00:00
